In [6]:
import os
import cv2 as cv
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
import tensorflow as tf

In [7]:
def load_images_from_folder(foldername, target_size=(64, 64)):
    images = []
    for filename in os.listdir(foldername):
        img = cv.imread(os.path.join(foldername, filename), cv.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv.resize(img, target_size)
            images.append(img)
    return images

def PSNR(original, compressed):
    original = tf.cast(original, tf.float32)
    compressed = tf.cast(compressed, tf.float32)      
    mse = tf.reduce_mean(tf.square(original - compressed))
    max_pixel = 255.0
    psnr = 20 * tf.math.log(max_pixel / tf.sqrt(mse)) / tf.math.log(10.0)
    return psnr.numpy()   # return numpy float here

In [8]:
# Define encoder and autoencoder models
def encode(input_img):
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    return encoded

def forward(input_img):
    encoded = encode(input_img)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    return decoded

input_img = Input(shape=(64, 64, 1))
autoencoder = Model(input_img, forward(input_img))
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

encoder_model = Model(input_img, encode(input_img))


In [9]:
# Load and preprocess images
images = load_images_from_folder("pneumonia")
images = np.array(images, dtype=np.float32) / 255.0
images = np.expand_dims(images, axis=-1)  # add channel dimension

def add_poisson_noise(images, lam):
    noisy = images + np.random.poisson(lam, images.shape) / 255.0
    return np.clip(noisy, 0., 1.)

# Add noise
lam25 = add_poisson_noise(images, 25)
lam50 = add_poisson_noise(images, 50)
lam75 = add_poisson_noise(images, 75)

In [10]:
# Split dataset for training/testing
x_train, x_test = train_test_split(images, test_size=0.2, random_state=42)
train25, test25 = train_test_split(lam25, test_size=0.2, random_state=42)
train50, test50 = train_test_split(lam50, test_size=0.2, random_state=42)
train75, test75 = train_test_split(lam75, test_size=0.2, random_state=42)

In [11]:
# Train and evaluate autoencoders on noisy images
noisy_lambdas = [25, 50, 75]
train_sets = [train25, train50, train75]
test_sets = [test25, test50, test75]

for lam, train_data, test_data in zip(noisy_lambdas, train_sets, test_sets):
    print(f"--- Lambda = {lam} ---")
    model = Model(input_img, forward(input_img))
    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(train_data, x_train,
              epochs=5,
              batch_size=128,
              shuffle=True,
              validation_data=(test_data, x_test),
              verbose=2)

    decoded_test = model.predict(test_data)

    psnr_scores = np.array([PSNR(x, y) for x, y in zip(test_data, decoded_test)])  
    print(f"Average PSNR after training on noisy inputs (lambda={lam}): {psnr_scores.mean():.4f}")

    encoded_train_noisy = encode(train_data)
    encoded_flat = encoded_train_noisy.numpy().reshape((encoded_train_noisy.shape[0], -1))  # convert to numpy and flatten
    np.savetxt(f"encoded_images_lam{lam}.csv", encoded_flat, delimiter=",")

--- Lambda = 25 ---
Epoch 1/5
27/27 - 12s - 461ms/step - loss: 0.0302 - val_loss: 0.0115
Epoch 2/5
27/27 - 11s - 423ms/step - loss: 0.0078 - val_loss: 0.0059
Epoch 3/5
27/27 - 11s - 424ms/step - loss: 0.0055 - val_loss: 0.0045
Epoch 4/5
27/27 - 12s - 443ms/step - loss: 0.0043 - val_loss: 0.0038
Epoch 5/5
27/27 - 12s - 443ms/step - loss: 0.0042 - val_loss: 0.0037
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
Average PSNR after training on noisy inputs (lambda=25): 67.6918
--- Lambda = 50 ---
Epoch 1/5
27/27 - 18s - 661ms/step - loss: 0.0333 - val_loss: 0.0127
Epoch 2/5
27/27 - 13s - 482ms/step - loss: 0.0100 - val_loss: 0.0079
Epoch 3/5
27/27 - 22s - 818ms/step - loss: 0.0067 - val_loss: 0.0058
Epoch 4/5
27/27 - 18s - 664ms/step - loss: 0.0052 - val_loss: 0.0050
Epoch 5/5
27/27 - 17s - 622ms/step - loss: 0.0045 - val_loss: 0.0041
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step
Average PSNR after training on noisy inputs (lambda=50): 61.8021
--- Lambda = 75 ---
Epoch 1/5
27/27 - 16s - 597ms/step - los